In [0]:
%sql
select
        desp.id_deputado,
        desp.ano,
        desp.cnpj_cpf_fornecedor,
        desp.mes,
        desp.tipo_despesa,
        desp.tipo_documento,
        CONCAT(desp.ano, LPAD(desp.mes, 2, '0')) as ano_mes,
        cad.sigla_partido,
        CURRENT_TIMESTAMP() as data_processamento_fato,
        round(sum(desp.valor_documento),2) as valor_documento,
        round(sum(desp.valor_liquido),2) as valor_liquido,
        round(sum(desp.valor_glosa),2) as valor_glosa,
        count(distinct desp.num_documento) as num_documentos
    FROM workspace.deputados.silver_despesas_deputados as desp
    JOIN workspace.deputados.silver_cadastro_deputados as cad
      ON cad.id = desp.id_deputado
    group by desp.id_deputado, desp.ano, desp.cnpj_cpf_fornecedor, desp.mes, desp.tipo_despesa, desp.tipo_documento, desp.ano_mes, cad.sigla_partido, cad.sigla_partido
    order by desp.id_deputado, desp.ano asc

In [0]:
df = spark.sql("""
    select
        desp.id_deputado,
        desp.ano,
        desp.cnpj_cpf_fornecedor,
        desp.mes,
        desp.tipo_despesa,
        desp.tipo_documento,
        CONCAT(desp.ano, LPAD(desp.mes, 2, '0')) as ano_mes,
        cad.sigla_partido,
        CURRENT_TIMESTAMP() as data_processamento_fato,
        round(sum(desp.valor_documento),2) as valor_documento,
        round(sum(desp.valor_liquido),2) as valor_liquido,
        round(sum(desp.valor_glosa),2) as valor_glosa,
        count(distinct desp.num_documento) as num_documentos
    FROM workspace.deputados.silver_despesas_deputados as desp
    JOIN workspace.deputados.silver_cadastro_deputados as cad
      ON cad.id = desp.id_deputado
    group by desp.id_deputado, desp.ano, desp.cnpj_cpf_fornecedor, desp.mes, desp.tipo_despesa, desp.tipo_documento, desp.ano_mes, cad.sigla_partido, cad.sigla_partido
    order by desp.id_deputado, desp.ano asc
""")

df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save("/Volumes/workspace/deputados/deputados/gold/fato_despesas_dep")

print("✅ Tabela gravada com sucesso!")

In [0]:
# Registra como tabela no catalog (para usar com SQL)
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.deputados.gold_fato_despedas_deputados")

print("✅ Tabela 'gold_fato_despedas_deputados' registrada no catalog!")